In [ ]:
import pandas as pd
import numpy as np
import re
import os
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import grangercausalitytests, adfuller
from statsmodels.tsa.vector_ar.vecm import coint_johansen

projects = ['neo4j', 'tomcat', 'jitsi',
'eclipse_jdt_core', 'spring-security', 'Arduino',
'eclipse_pde_ui', 'hibernate-orm', 'jabref', 'jenkins',
'pmd', 'rt_equinox_framework']

metrics = ['loc', 'noc', 'fanin', 'fanout', 'dit']

In [ ]:
loc_df = pd.read_csv('./metrics/loc.csv', index_col=0)
loc_df.index = [item + '_loc' for item in loc_df.index]

noc_df = pd.read_csv('./metrics/noc.csv', index_col=0)
noc_df.index = [item + '_noc' for item in noc_df.index]

log_df = pd.read_csv('./metrics/logStatementsQty.csv', index_col=0)
log_df.index = [item + '_log' for item in log_df.index]

try_df = pd.read_csv('./metrics/tryCatchQty.csv', index_col=0)
try_df.index = [item + '_try' for item in try_df.index] 


fanin_df = pd.read_csv('./metrics/fanin.csv', index_col=0)
fanin_df.index = [item + '_fanin' for item in fanin_df.index]

fanout_df = pd.read_csv('./metrics/fanout.csv', index_col=0)
fanout_df.index = [item + '_fanout' for item in fanout_df.index]

dit_df = pd.read_csv('./metrics/dit.csv', index_col=0)
dit_df.index = [item + '_dit' for item in dit_df.index]

lcc_df = pd.read_csv('./metrics/lcc.csv', index_col=0)
lcc_df.index = [item + '_lcc' for item in lcc_df.index]

In [ ]:
dit_df

In [ ]:
## normalize all dataframes to 0-1
def normalize_df(df):
    return (df - df.min()) / (df.max() - df.min())

log_df = normalize_df(log_df)
try_df = normalize_df(try_df)  
loc_df = normalize_df(loc_df)
noc_df = normalize_df(noc_df)
fanin_df = normalize_df(fanin_df)
fanout_df = normalize_df(fanout_df)
dit_df = normalize_df(dit_df)

In [ ]:
full_df = pd.concat([loc_df, noc_df, fanin_df, fanout_df, dit_df], axis=0)

In [ ]:
full_df.dropna(thresh=300, inplace=True)
full_df.dropna(axis=1, inplace=True)

In [ ]:
projects_dict = {}
p_values_dict = {}

for project in projects:
    project_p_values = []
    project_dfs = []
    for metric in metrics:
        metric_df = full_df.T[[project + '_' + metric]]
        metric_df.columns = [metric]
        
        adfuller_result = adfuller(metric_df.values)
        
        if adfuller_result[1] >= 0.05:
            metric_df = metric_df.diff()
                
        ## drop first row
        metric_df = metric_df[1:]
        project_dfs.append(metric_df)
        project_p_values.append(adfuller_result[1])
        
    projcet_agg_df = pd.concat(project_dfs, axis=1)

    p_values_dict[project] = project_p_values
    projects_dict[project] = projcet_agg_df
    
    
# Create a dataframe from the p-values dictionary
p_values_df = pd.DataFrame(p_values_dict, index=metrics)

# Split the dataframe into two halves
half = len(p_values_df.columns) // 2
df1 = p_values_df.iloc[:, :half]
df2 = p_values_df.iloc[:, half:]

# Plot both tables in the same plot
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 20))

# Plot the first half of the table
ax1.axis('off')
ax1.table(cellText=df1.values, colLabels=df1.columns, rowLabels=df1.index, loc='center', cellLoc='center')
ax1.set_title('P-values for Adfuller test for each metric in each project')

# Plot the second half of the table
ax2.axis('off')
ax2.table(cellText=df2.values, colLabels=df2.columns, rowLabels=df2.index, loc='center', cellLoc='center')

## save plot
plt.savefig('./plots/adfuller_pvalues.png', dpi=300, bbox_inches='tight')


## create csv file with p-values with 2 decimal points
p_values_df = p_values_df.round(2)
p_values_df.reset_index(inplace=True)
p_values_df.to_csv('./tables/adfuller_pvalues.csv', index=True)

In [ ]:
try_df.dropna(thresh=300, inplace=True)
try_df.dropna(axis=1, inplace=True)

try_df.drop(['pentaho-kettle_try'], axis=0, inplace=True)


In [ ]:
log_df.dropna(thresh=300, inplace=True)
log_df.dropna(axis=1, inplace=True)

log_df.drop(['pentaho-kettle_log'], axis=0, inplace=True)

In [ ]:
for project in projects:
    project_df = try_df.loc[project + '_try']
    
    
    adfuller_result = adfuller(project_df.values)
    
    if adfuller_result[1] >= 0.05:
        project_df = project_df.diff()
        
    try_df.loc[project + '_try'] = project_df[1:]
    
try_df = try_df.iloc[:,1:]

In [ ]:
for project in projects:
    project_df = log_df.loc[project + '_log']
    
    
    adfuller_result = adfuller(project_df.values)
    
    if adfuller_result[1] >= 0.05:
        project_df = project_df.diff()
        
    log_df.loc[project + '_log'] = project_df[1:]
    
log_df = log_df.iloc[:,1:]

In [ ]:
def grangers_causation_matrix(data, variables, maxlag, test='ssr_ftest'):    
    """Check Granger Causality of all possible combinations of the Time series.
    The rows are the response variable, columns are predictors. The values in the table 
    are the P-Values. P-Values lesser than the significance level (0.05), implies 
    the Null Hypothesis that the coefficients of the corresponding past values is 
    zero, that is, the X does not cause Y can be rejected.

    data      : pandas dataframe containing the time series variables
    variables : list containing names of the time series variables.
    """
    df = pd.DataFrame(np.zeros((len(variables), len(variables))), columns=variables, index=variables)
    for c in df.columns:
        for r in df.index:
            test_result = grangercausalitytests(data[[r, c]], maxlag=maxlag, verbose=False)
            p_values = [round(test_result[i+1][0][test][1],4) for i in range(maxlag)]
            # if verbose: print(f'Y = {r}, X = {c}, P Values = {p_values}')
            min_p_value = np.min(p_values)
            df.loc[r, c] = min_p_value
    df.columns = [var + '_x' for var in variables]
    df.index = [var + '_y' for var in variables]
    return df

In [ ]:
corr_matrices = {}

for proj in projects:

    corr_df = pd.concat([projects_dict[proj], try_df.loc[proj + '_try'], log_df.loc[proj + '_log']], axis=1)

    ## drop all constant columns and keep track of them
    corr_df = corr_df.loc[:, (corr_df != corr_df.iloc[0]).any()]

    corr_matrix = grangers_causation_matrix(corr_df, variables = corr_df.columns, maxlag=3)


    corr_matrix = corr_matrix[[proj + '_try_x',proj + '_log_x']].drop([proj + '_try_y',proj + '_log_y'])
    
    corr_matrix.columns = ['try', 'log']
    corr_matrix.index = ['loc', 'noc', 'fanin', 'fanout', 'dit']
    
    corr_matrices[proj] = corr_matrix
    
    


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

for proj in projects:
    # Define the correlation matrix
    corr_matrix = corr_matrices[proj]

    # Set the color thresholds
    cmap = sns.diverging_palette(220, 10, as_cmap=True)
    vmin = -1
    vmax = 1

    # Plot the heatmap
    plt.figure(figsize=(10, 8))
    sns.heatmap(corr_matrix, cmap=cmap, vmin=0, vmax=1, annot=True, fmt=".2f", linewidths=0.5)
    plt.title(f"P-values for Granger Causality Test: {proj}")
    ## save plot
    plt.savefig(f'./plots/{proj}_corr.png', dpi=300)
    plt.close()

## create csv file with p-values for each project
for proj in projects:
    corr_matrices[proj].reset_index().to_csv(f'./tables/{proj}_corr.csv', index=False)


In [ ]:
def sumdf(df, project):
    
    df = df[df != -1].sum()

    df = pd.DataFrame(df).T

    df.index = [project]
    df.columns = list(np.arange(1, len(df.columns)+1))
    
    return df

def iqrdf(df, project):
    q1 = df[df!=-1].quantile(0.25)
    q3 = df[df!=-1].quantile(0.75)
    iqr = q3 - q1

    df = pd.DataFrame(iqr).T

    df.index = [project]
    df.columns = list(np.arange(1, len(df.columns)+1))
    
    return df

def meandf(df, project):
    
    df = df[df != -1].mean()

    df = pd.DataFrame(df).T

    df.index = [project]
    df.columns = list(np.arange(1, len(df.columns)+1))
    
    return df

def mediandf(df, project):
    
    df = df[df != -1].median()

    df = pd.DataFrame(df).T

    df.index = [project]
    df.columns = list(np.arange(1, len(df.columns)+1))
    
    return df

def maxdf(df, project):
    
    df = df[df != -1].max()

    df = pd.DataFrame(df).T

    df.index = [project]
    df.columns = list(np.arange(1, len(df.columns)+1))
    
    return df

In [ ]:
for metric in ['loc', 'noc', 'tryCatchQty', 'logStatementsQty']:

    all = []
    for project in os.listdir('./data/'):

        try:
            df = pd.read_csv(f'./data/{project}/{metric}.csv', header=None)
            df.set_index(0, inplace=True)
        except:
            continue

        df = df.query('~index.str.lower().str.contains("test")')

        df = sumdf(df, project)
        
        all.append(df)
        
    new = pd.concat(all, axis=0)
    new.to_csv(f'./metrics/{metric}.csv', index=True)

In [ ]:
for metric in ['dit', 'fanin', 'fanout', 'lcc']:
 
    all = []
    for project in os.listdir('./data/'): 
        
        try:
            df = pd.read_csv(f'./data/{project}/{metric}.csv', header=None)
        except:
            continue   
        df.set_index(0, inplace=True)

        df = df.query('~index.str.lower().str.contains("test")')
        df = iqrdf(df, project)
        
        all.append(df)
        
    new = pd.concat(all, axis=0)
    new.to_csv(f'./metrics/{metric}.csv', index=True)

In [ ]:
# for metric in ['dit', 'fanin', 'fanout', 'lcc']:
for metric in ['dit']:
 
    all = []
    for project in os.listdir('./data/'): 
        
        try:
            df = pd.read_csv(f'./data/{project}/{metric}.csv', header=None)
        except:
            continue   
        df.set_index(0, inplace=True)

        df = df.query('~index.str.lower().str.contains("test")')
        df = meandf(df, project)
        
        all.append(df)
        
    new = pd.concat(all, axis=0)
    new.to_csv(f'./metrics/{metric}_mean.csv', index=True)

In [ ]:
for metric in ['dit', 'fanin', 'fanout']:
 
    all = []
    for project in os.listdir('./data/'): 
        
        try:
            df = pd.read_csv(f'./data/{project}/{metric}.csv', header=None)
        except:
            continue   
        df.set_index(0, inplace=True)

        df = df.query('~index.str.lower().str.contains("test")')
        df = mediandf(df, project)
        
        all.append(df)
        
    new = pd.concat(all, axis=0)
    new.to_csv(f'./metrics/{metric}_median.csv', index=True)

In [ ]:
for metric in ['dit', 'fanin', 'fanout']:
 
    all = []
    for project in os.listdir('./data/'): 
        
        try:
            df = pd.read_csv(f'./data/{project}/{metric}.csv', header=None)
        except:
            continue   
        df.set_index(0, inplace=True)

        df = df.query('~index.str.lower().str.contains("test")')
        df = maxdf(df, project)
        
        all.append(df)
        
    new = pd.concat(all, axis=0)
    new.to_csv(f'./metrics/{metric}_max.csv', index=True)